<a href="https://colab.research.google.com/github/hida0/ESAA-2022-2/blob/HW/Week08_HW1_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 교재 필사
(파이썬 머신러닝 완벽 가이드 ch8. 5 pg. 497~512)

### 5. 감성 분석
- **감성 분석(Sentiment Analysis)**은 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용된다.
- 감성 분석은 **문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산**하는 방법을 이용한다. 이러한 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산하여 긍정 감성 또는 부정 감성을 결정한다.
- 이러한 감성 분석은 머신러닝 관점에서 **지도학습과 비지도학습** 방식으로 나눌 수 있다.
 - **지도학습**은 **학습 데이터와 타겟 레이블 값을 기반으로 감성 분석 학습**을 수행한 뒤, **이를 기반으로 다른 데이터의 감성 분석을 예측**하는 일반적인 텍스트 기반의 분류와 거의 동일하다.
 - **비지도학습**은 **'Lexicon'**이라는 일종의 **감성 어휘 사전을 이용**한다. Lexicon은 **감성 분석을 위한 용어와 문맥에 대한 다양한 정보**를 가지고 있으며, 이를 이용하여 **문서의 긍정적, 부정적 감성 여부를 판단**한다.

### 5-1. 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 지도학습 기반 감성 분석은 텍스트 기반의 이진 분류라고도 표현할 수 있다.
- 유명한 IMDB 영화 사이트의 영화평을 이용하여, 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델을 만든다.
- 주어진 데이터는 탭(\t)으로 분리된 파일로, 판다스의 `read_csv( )`를 이용하면 인자로 `sep='\t'`를 명시하여 로드할 수 있다.
- 사용되는 피처는 다음과 같다.
 - id: 각 데이터의 id
 - sentiment: 영화평(review)의 Sentiment 결과값(Target Label). **1은 긍정적 평가, 0은 부정적 평가를 의미**
 - review: 영화평의 텍스트

In [ ]:
import pandas as pd
review_df = pd.read_csv('/content/drive/MyDrive/ESAA/data/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [ ]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

### 5-1. 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 여전히 <br \>가 존재하며, 이 문자열은 피처로 만들 필요가 없으므로 삭제한다.
- 판다스의 DataFrame과 Series는 문자열 연산을 지원하기 위해 **`str`** 속성을 이용한다. **DataFrame/Series 객체에서 `str`을 적용하면 다양한 문자열 연산을 수행**할 수 있다. 
- **`str`에  `replace( )`를 적용**하여 **<br \> 태그를 공백으로** 모두 바꾼다.
- **영어가 아닌 숫자와 특수문자** 역시 Sentiment를 위한 피처로는 별 의미가 없어 보이므로 이들도 **모두 공백으로 변경**한다. 숫자/특수문자를 찾고 이를 **변환하는 것은 정규 표현식을 이용**한다.
- 파이썬의 **`re`** 모듈은 편리하게 정규 표현식을 지원한다. 정규 표현식 **[^a-zA-Z]의 의미는 영어 대/소문자가 아닌 모든 문자를 찾는 것**이다.
- **`re.sub('[^a-zA-Z]', ' ', x)`**는 **영어 대/소문자가 아닌 모든 문자를 찾아서 공백으로 변경**한다. `lambda` 식을 이용하여 적용한다.
---
- 결정값 클래스인 sentiment 칼럼을 별도로 추출하여 결정값 데이터 세트를 만들고, 원본 데이터에서 id와 sentiment 칼럼을 삭제하여 피처 데이터 세트를 생성한다.
- `train_test_split( )`을 이용하여 학습용과 테스트용 데이터 세트로 분리한다. 테스트용 데이터 세트의 비율을 30%로 지정한다.
- 학습용 데이터는 17500개의 리뷰, 테스트용 데이터는 7500개의 리뷰로 구성된다.

In [ ]:
#<br> html 태그는 replace 함수를 이용하여 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br \>', ' ')

import re
#파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [ ]:
class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
print(X_train.shape, X_test.shape)

(17500, 1) (7500, 1)


### 5-1. 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 감상평(Review) 텍스트를 **피처 벡터화**한 후에 **ML 분류 알고리즘을 적용하여 예측 성능을 측정**한다. **`Pipeline`** 객체를 이용하여 이 두가지를 한꺼번에 수행한다.
- 먼저 **Count 벡터화**를 적용하여 예측 성능을 측정하고, 다음으로 **TF-IDF 벡터화**를 적용한다.
- ML 분류 알고리즘으로는 **`LogisticRegression`**을 이용한다.
- 예측 성능 평가는 이진 분류임을 고려하여 **테스트 데이터 세트의 정확도와 ROC-AUC**를 모두 측정한다.
- **`CountVectorizer`**로 Count 벡터화를 적용하여 예측 성능을 측정한다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

#stopwords는 English, filtering, ngram은 (1, 2)로 설정하여 CountVectorization 수행
#LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

#Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probas = pipeline.predict_proba(X_test['review'])[:, 1]
print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4}".format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probas)))

예측 정확도는 0.8865, ROC-AUC는 0.9506


### 5-1. 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 이번에는 **`TfidfVectorizer`**로 TF-IDF 벡터화를 적용하여 다시 예측 성능을 측정한다.
- 코드는 위와 거의 동일하며, Pipeline에서 CountVectorizer를 TfidfVectorizer로 변경하면 된다.
- 예측 정확도와 ROC-AUC 모두 Count 기반 피처 벡터화보다 TF-IDF 기반 피처 벡터화의 예측 성능이 조금 더 향상되었다.

In [ ]:
#stopwords는 English, filtering, ngram은 (1, 2)로 설정하여 TF-IDFVectorization 수행
#LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

#Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probas = pipeline.predict_proba(X_test['review'])[:, 1]
print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4}".format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probas)))

예측 정확도는 0.8932, ROC-AUC는 0.96


### 5-2. 비지도학습 기반 감성 분석
- 지도 감성 분석은 데이터 세트가 레이블 값을 가지고 있다. 하지만 많은 감성 분석용 데이터는 이러한 **결정된 레이블 값을 가지고 있지 않다**.
- 이러한 경우에 수행하는 **비지도 감성 분석은 Lexicon을 기반**으로 하며 비록 한글을 지원하지 않지만 유용하게 사용된다. 
- **Lexicon은** 일반적으로 어휘집을 의미하지만 여기서는 **주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전**이다. 
- 감성 사전은 긍정 감성 또는 부정 감성 정도를 의미하는 수치를 가지고 있으며, 이를 **감성 지수**(Polarity score)라고 한다. 감성 지수는 단어의 위치나 주변 단어, 문맥 등을 참고하여 결정된다.
- 이러한 **감성 사전을 구현한 대표격은 `NLTK` 패키지**이다. 이것은 많은 서브 모듈을 가지고 있으며 그 중에 감성 사전인 **`Lexicon`** 모듈도 포함되어 있다.
---
- **`NLP`**에서 제공하는 **`WordNet`** 모듈은 **방대한 영어 어휘 사전**으로, 단순한 어휘 사전이 아닌 **시맨틱 분석을 제공**하는 어휘 사전이다.
- 텍스트 분석을 공부하면 '시맨틱(semantic)'이라는 용어를 자주 접하게 되는 이는 '문맥상 의미'를 의미한다. 동일한 단어나 문장이라도 다른 환경과 문맥에서는 다르게 표현되거나 이해될 수 있다.
- 언어학에서는 이러한 시맨틱을 표현하기 위해 여러 가지 규칙을 정해왔으며, NLP 패키지는 시맨틱을 프로그램적으로 인터페이스할 수 있는 다양한 방법을 제공한다.
- **WordNet은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공**하며, 이를 위해 **각각의 품사로 구성된 개별 단어를 Synset이라는 개념을 이용해 표현**한다. **Synset**은 단순한 하나의 단어가 아니라 **그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념**이다.

### 5-2. 비지도학습 기반 감성 분석
- **`NLTK`의 감성 사전이 감성에 대한 훌륭한 사전 역할을 제공**한다는 장점이 있지만, 아쉽게도 **예측 성능은 그리 좋지 못하다**는 단점도 있다.  그리하여 실제 업무의 적용은 NLTK 패키지가 아닌 다른 감성 사전을 적용하는 것이 일반적이다.
- NLTK를 포함한 **대표적인 감성 사전**은 다음과 같다.
 - **SentiWordNet**: NLTK 패키지의 WordNet과 유사하게 **감성 단어 전용의 WordNet을 구현**한 것으로, **WordNet의 Synset 개념을 감성 분석에 적용한 것**이다. **WordNet의 Synset별로 3가지 감성 점수(긍정 감성 지수, 부정 감성 지수, 객관성 지수)를 할당**한다. 긍정 감성 지수는 해당 단어가 감성적으로 얼마나 긍정적인가를, 부정 지수는 얼마나 감성적으로 부정적인가를 수치로 나타낸 것이다. 객관성 지수는 긍정/부정 감성 지수와 완전히 반대되는 개념으로 단어가 감성과 관계없이 얼마나 객관적인지를 수치로 나타낸 것이다. **문장별로 단어의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산**하고, 이에 기반하여 **감성이 긍정인지 부정인지를 결정**한다.
 - **VADER**: **주로 소셜 미디어의 텍스트에 대한 감성 분석**을 제공하기 위한 패키지이다. **뛰어난 감성 분석 결과를 제공**하며, **비교적 빠른 수행 시간**을 보장해 대용량 텍스트 데이터에 잘 사용되는 패키지이다.
 - **Pattern**: **예측 성능** 측면에서 가장 주목받는 패키지이다.
---
- **`SentiWordNet`**과 **`VADER`** 감성 사전을 이용하여 감성 분석을 수행한 뒤 예측 성능을 지도학습 기반의 분류와 비교한다.

### 5-3. SentiWordNet을 이용한 감성 분석
#### 1) WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해
- SentiWordNet은 WordNet 기반의 Synset을 이용하므로 먼저 Synset에 대한 개념을 알아본다.
- **`WordNet`**을 이용하기 위해서는 **`NLTK`**를 셋업한 후에 WordNet 서브패키지와 데이터 세트를 내려받아야 한다. **`download('all')`**로 모든 데이터 세트와 패키지를 내려받는다.
- NLTK의 모든 데이터 세트를 내려받은 뒤에 **`WordNet`** 모듈을 임포트해서 'present' 단어에 대한 Synset을 추출한다. **`WordNet`**의 **`synsets( )`**는 **파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체를 반환**한다.
---
- **`synsets( )`** 호출 시 반환되는 것은 **여러 개의 Synset 객체를 가지는 리스트**이다. 총 18개의 서로 다른 시맨틱을 가지는 Synset 객체가 반환되었다.
- Synset('present.n.01')와 같이 Synset 객체의 파라미터 'present.n.01'은 POS 태그를 나타낸다. 'present.n.01'에서 **present는 의미**, **n은 명사 품사**, **01은 present가 명사로서 가지는 의미**가 여러개이므로 **이를 구분하는 인덱스**이다.


In [ ]:
import nltk
nltk.download('all')

In [ ]:
from nltk.corpus import wordnet as wn
term='present'

#'present'라는 단어로 WordNet의 synsets 생성
synsets = wn.synsets(term)
print('synsets( ) 반환 type:', type(synsets))
print('synsets( ) 반환 값 개수:', len(synsets))
print('synsets( ) 반환 값:' , synsets)

synsets( ) 반환 type: <class 'list'>
synsets( ) 반환 값 개수: 18
synsets( ) 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


### 5-3. SentiWordNet을 이용한 감성 분석
#### 1) WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해
- Synset 객체가 가지는 여러 가지 속성을 살펴본다. **`Synset`**은 **POS(Part of Speech, 우리말로 하면 품사), 정의(Definition), 부명제(Lemma)** 등으로 **시맨틱적인 요소를 표현**할 수 있다.
- **Synset('present.n.01')**과 **Synset('present.n.02')**는 명사지만 서로 다른 의미를 가진다.
 - Synset('present.n.01')는 POS가 noun.time이며 Definition을 살펴보면 '시간적인 의미로 현재'를 나타낸다. 
 - Synset('present.n.02')는 POS가 noun.possession이며 Definition은  '선물'이다.
- Synset('show.v.01')은 POS가 verb.perception이며 Definition은 '관객에게 전시물 등을 보여주다'라는 뜻이다. 
- 이처럼 **`Synset`**은 **하나의 단어가 가질 수 있는 여러 가지 시맨틱 정보를 개별 클래스로 나타낸 것**이다.







In [ ]:
for synset in synsets:
    print("##### Synset name: ", synset.name(), "#####")
    print("POS: ", synset.lexname())
    print("Definition: ", synset.definition())
    print("Lemmas: ", synset.lemma_names())

##### Synset name:  present.n.01 #####
POS:  noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas:  ['present', 'nowadays']
##### Synset name:  present.n.02 #####
POS:  noun.possession
Definition:  something presented as a gift
Lemmas:  ['present']
##### Synset name:  present.n.03 #####
POS:  noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemmas:  ['present', 'present_tense']
##### Synset name:  show.v.01 #####
POS:  verb.perception
Definition:  give an exhibition of to an interested audience
Lemmas:  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name:  present.v.02 #####
POS:  verb.communication
Definition:  bring forward and present to the mind
Lemmas:  ['present', 'represent', 'lay_out']
##### Synset name:  stage.v.01 #####
POS:  verb.creation
Definition:  perform (a play), especially on a stage
Lemmas:  ['stage', 'present', 're

### 5-3. SentiWordNet을 이용한 감성 분석
#### 1) WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해
- WordNet은 **어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있다**. 
- **`Synset`** 객체는 **단어 간의 유사도**를 나타내기 위해 **`path_similarity( )` 메서드를 제공**한다.
- `path_similarity( )`를 이용하여 'tree', 'lion', 'tiger', 'cat', 'dog' 라는 단어의 상호 유사도를 살펴본다.
- lion은 tree와의 상호 유사도가 0.07로 가장 작고, tiger와의 상호 유사도가 0.33으로 가장 크다.

In [ ]:
#synset 객체를 단어별로 생성
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

In [ ]:
entities = [tree, lion, tiger, cat, dog] #synset 객체
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]
#entities[0].name() : 'tree.n.01' 
#entities[0].name().split('.') : ['tree', 'n', '01']

#단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
    similarities.append(similarity)

#개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame으로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


### 5-3. SentiWordNet을 이용한 감성 분석
#### 1) WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해
- **SentiWordNet**은 WordNet의 Synset과 유사한 **Senti_Synset클래스**를 가지고 있다.
- **`Sentiwordnet`** 모듈의 **`senti_synsets( )`**는 WordNet 모듈이라서 `synsets( )`와 비슷하게 **Senti_Synset 클래스를 리스트 형태로 반환**한다.
---
- SentiWordNet 객체는 **단어의 감성을 나타내는 감성 지수**와 **객관성을 나타내는 객관성 지수**를 가지고 있다. 
- 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1이 되고, 감성 지수는 모두 0이 된다. 감성 지수는 다시 긍정 감성 지수와 부정 감성 지수로 나뉜다.
- 다음은 father(아버지)라는 단어와 fabulous(아주 멋진)라는 2개 단어의 감성 지수와 객관성 지수를 알아본 것이다.
- 원하는 단어에 대하여 생성한 SentiWordNet 객체에 긍정 감성 지수는 **`pos_score( )`**, 부정 감성 지수는 **`neg_score( )`**, 객관성 지수는 **`obj_score( )`**를 이용하여 확인할 수 있다.
- father는 객관적인 단어로 객관성 지수가 1.0이고, 긍정/부정 감성 지수는 모두 0이다. 반면 fabulous는 감성 단어로 긍정 감성 지수가 0.875, 부정 감성 지수가 0.125이다.

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

#'slow'라는 단어로 SentiWordNet의 synsets 생성
senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets( ) 반환 type:', type(senti_synsets))
print('senti_synsets( ) 반환 값 개수:', len(senti_synsets))
print('senti_synsets( ) 반환 값:' , senti_synsets)

senti_synsets( ) 반환 type: <class 'list'>
senti_synsets( ) 반환 값 개수: 11
senti_synsets( ) 반환 값: [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [ ]:
father = swn.senti_synset('father.n.01')
print("father 긍정 감성 지수: ", father.pos_score())
print("father 부정 감성 지수: ", father.neg_score())
print("father 객관성 지수: ", father.obj_score())

father 긍정 감성 지수:  0.0
father 부정 감성 지수:  0.0
father 객관성 지수:  1.0


In [ ]:
fabulous = swn.senti_synset('fabulous.a.01')
print("fabulous 긍정 감성 지수: ", fabulous.pos_score())
print("fabulous 부정 감성 지수: ", fabulous.neg_score())

fabulous 긍정 감성 지수:  0.875
fabulous 부정 감성 지수:  0.125


### 5-3. SentiWordNet을 이용한 감성 분석
#### 2) SentiWordNet을 이용한 영화 감상평 감성 분석
- 앞의 예제에서 적용한 IMDB 영화 감상평 분석을 SentiWordNet Lexicon 기반으로 수행한다.
- SentiWordNet을 이용해 감성 분석을 수행하는 개략적인 순서는 다음과 같다.
 1. 문서(Document)를 문장(Sentence) 단위로 분해
 2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
 3. 품사 태깅된 단어 기반으로 Synset 객체와 Senti_synset 객체를 생성
 4. Senti_synset 객체에서 긍정/부정 감성 지수를 구하고, 이를 모두 합산하여 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때 부정 감성으로 결정
- SentiWordNet을 이용하기 위해서 **WordNet을 이용해 문서를 다시 단어로 토큰화**한 뒤 **어근 추출(Lemmatization)과 품사 태깅(POS Tagging)**을 적용해야 한다.
---
- 먼저 **품사 태깅을 수행**하는 내부 함수를 생성한다.
- 다음으로 **(문서 -> 문장 -> 단어)로 토큰 -> 품사 태깅** 후에 **SentiSynset 클래스를 생성하고 감성 지수를 합산**하는 함수를 생성한다. 각 단어의 긍정 감성 지수와 부정 감성 지수를 모두 합한 **총 감성 지수가 0 이상일 경우 긍정 감성으로, 그렇지 않을 경우 부정 감성으로 예측**한다.

In [ ]:
from nltk.corpus import wordnet as wn

#간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    #감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer() 

    #분해된 문장별로 단어 토큰 -> 품사 태깅 후 SentiSynset 생성 -> 감성 지수 합산
    raw_sentences = sent_tokenize(text) #문장 분해
    for raw_sentence in raw_sentences:
        #NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence:
            #WordNet 기반의 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            #sentiwordnt의 감성 단어 분석으로 감성 synset 추출
            #모든 단어에 대해 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산하여 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score()-swn_synset.neg_score())
            tokens_count += 1

        if not tokens_count:
            return 0
        
        #총 score가 0 이상일 경우 긍정 감성으로 1, 그렇지 않을 경우 부정 감성으로 0 반환
        if sentiment >= 0:
            return 1

        return 0

### 5-3. SentiWordNet을 이용한 감성 분석
#### 2) SentiWordNet을 이용한 영화 감상평 감성 분석
- 생성한 함수를 IMDB 감상평의 개별 문서에 적용하여 긍정 및 부정 감성을 예측한다.
- 판다스의 `apply lambda` 구문을 이용해 swn_polarity(text)를 개별 감상평 텍스트에 적용한다. 반환된 감성 평가를 review_df의 새로운 칼럼 'preds'로 추가한다.
- 실제 감성 평가인 'sentiment'과 예측 결과인 'preds'에 대하여 정확도, 정밀도, 재현율 값을 측정한다.
- 정확도가 약 66.1%, 정밀도가 약 64.7%, 재현율이 약 67.64%이다. 

In [ ]:
review_df = pd.read_csv('/content/drive/MyDrive/ESAA/data/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df['review'] = review_df['review'].str.replace('<br \>', ' ')
review_df['review'] = review_df['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [ ]:
review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))

#실제값
y_target = review_df['sentiment'].values
#예측값
preds = review_df['preds'].values

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
print(confusion_matrix(y_target, preds))
print("정확도: ", np.round(accuracy_score(y_target, preds), 4))
print("정밀도: ", np.round(precision_score(y_target, preds), 4))
print("재현율: ", np.round(f1_score(y_target, preds), 4))

[[7669 4831]
 [3644 8856]]
정확도:  0.661
정밀도:  0.647
재현율:  0.6764


### 5-4. VADER을 이용한 감성 분석
- 또 다른 Lexicon인 **VADER Lexicon**이 있다. VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon이다. 
- VADER는 **`SentimentIntensityAnalyzer`** 클래스를 이용해 쉽게 감성 분석을 제공한다.
- VADER는 NLTK 패키지의 서브 모듈로 제공될 수도 있고, 단독 패키지로 제공될 수도 있다. 
- NLTK의 서브 모듈로 **`SentimentIntensityAnalyzer`**을 임포트하고, 간략하게 IMDB의 감상평 1개만 감성 분석을 수행하여 결과를 살펴본다.
---
- 먼저 **`SentimentIntensityAnalyzer`** 객체를 생성한 뒤에 문서별로 **`polarity_scores( )`** 메서드를 호출하여 **감성 점수**를 구한다. 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정 감성, 그렇지 않으면 부정 감성으로 판단한다.
-  SentimentIntensityAnalyzer 객체의 `polarity_scores( )` 메서드는 **딕셔너리 형태의 감성 점수를 반환**한다. 'neg'는 부정 감성 지수, 'neu'는 중립적 감성 지수, 'pos'는 긍정 감성 지수, 'compound'는 neg, neu, pos score를 적절하게 조합하여 -1에서 1 사이의 감성 지수를 표현한 값이다. 
- compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정한다. 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정 감성으로 판단하며 상황에 따라 임계값을 적절하게 조절하여 예측 성능을 조절한다.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.127, 'neu': 0.747, 'pos': 0.125, 'compound': -0.7943}


### 5-4. VADER을 이용한 감성 분석
- VADER를 이용해 IMDB 감성 분석을 수행하기 위해 vader_polarity( ) 함수를 생성한다. 이 함수는 **입력 파라미터로 영화 감상평 텍스트와 긍정/부정을 결정하는 임계값**을 가지며, **SentimentIntensityAnalyzer 객체의 `polarity_scores( )` 메서드를 호출하여 감성 결과를 반환**한다.
- review_df에 `apply lambda` 식을 이용해 vader_polarity( ) 함수를 호출하여 각 문서별로 구한 감성 결과를 새로운 칼럼 'vader_preds'로 추가한다. 그리고 저장된 감성 분석 결과를 기반으로 VADER의 예측 성능을 측정한다.
- 정확도가 약 69.57%, 정밀도가 약 64.92%, 재현율이 약 73.67%이다. (SentiWordNet을 이용한 결과: 정확도 약 66.1%, 정밀도 약 64.7%, 재현율 약 67.64%) 세 측정 지표에서 모두 상승했으며, 특히 재현율에서 많이 향상되었다.
- 감성 사전을 이용한 감성 분석 예측 성능은 지도학습 분류 기반의 예측 성능에 비하면 아직 낮은 수준이지만, 결정 클래스 값이 없는 상황을 고려한다면 어느 정도 만족할 만한 수준이다.

In [ ]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)

    #compound 값에 기반하여 threshold인 0.1보다 크면 긍정 감성으로 1, 그렇지 않으면 부정 감성으로 0을 반환
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [ ]:
#apply lambda 식을 이용하여 레코드별로 vader_polarity()를 수행하고 그 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x, 0.1))

y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print("정확도: ", np.round(accuracy_score(y_target, vader_preds), 4))
print("정밀도: ", np.round(precision_score(y_target, vader_preds), 4))
print("재현율: ", np.round(f1_score(y_target, vader_preds), 4))

[[ 6749  5751]
 [ 1856 10644]]
정확도:  0.6957
정밀도:  0.6492
재현율:  0.7367
